<a href="https://colab.research.google.com/github/DJLee68/MachineLearningProject/blob/master/Assignment09/DJLee_Assignment09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, utils, datasets

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)


In [2]:
 class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


In [3]:
criterion = nn.NLLLoss()
classifier = classification().to(GPU)
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value)

NameError: ignored

## Output using the dataset

1. Plot the original data points [1pt]

2. Plot the normalized data points [1pt]

3. Plot the principal axes [2pt]

4. Plot the first principal axis [3pt]

5. Plot the project of the normalized data points onto the first principal axis [4pt]

6. Plot the lines between the normalized data points and their projection points on the first principal axis [3pt]



7. Plot the second principal axis [3pt]

8. Plot the project of the normalized data points onto the second principal axis [4pt]

9. Plot the lines between the normalized data points and their projection points on the second principal axis [3pt]